## Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [64]:
import pandas as pd

In [65]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [66]:
ratings = ratings.groupby('movieId').mean()[['rating']].round(1).reset_index()
ratings.head()

,movieId,rating
0,1,3.9
1,2,3.4
2,3,3.2
3,4,2.4
4,5,3.3


In [67]:
def define_class(row):
    if row.rating <= 2:
        return 'низкий'
    elif row.rating < 4.5:
        return 'средний'
    else:
        return 'высокий'
        

In [68]:
ratings['class'] = ratings.apply(define_class,axis=1)
ratings.head()

,movieId,rating,class
0,1,3.9,средний
1,2,3.4,средний
2,3,3.2,средний
3,4,2.4,средний
4,5,3.3,средний


In [69]:
# Проверка, что присвоились разные рейтинги
ratings.groupby('class').count().reset_index()

,class,movieId,rating
0,высокий,672,672
1,низкий,979,979
2,средний,7415,7415


In [70]:
#соединяем два датафрэйма, для наглядности  и для повторения изученного
data_movies = ratings.merge(movies, on = 'movieId', how = 'left')
data_movies.head()


,movieId,rating,class,title,genres
0,1,3.9,средний,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,3.4,средний,Jumanji (1995),Adventure|Children|Fantasy
2,3,3.2,средний,Grumpier Old Men (1995),Comedy|Romance
3,4,2.4,средний,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,3.3,средний,Father of the Bride Part II (1995),Comedy


## Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [71]:
keywords = pd.read_csv('keywords.csv')
keywords

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
...,...,...
99995,эльдорадо старый оскол,3705
99996,frigate для firefox,3630
99997,укрсиб,3630
99998,погода в ялте на неделю,3688


In [72]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] }

In [73]:
def geo_class(row):
    for key,value in geo_data.items():
        i = 0
        while i < len(value):
            if value[i] in row.keyword:
                return key
            else:
                i += 1
    return 'undefined'  

In [74]:
keywords['region'] = keywords.apply(geo_class, axis = 1)
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [75]:
#Проверка, что распределение прошло по регионам
keywords.groupby('region').count()


,keyword,shows
region,,
undefined,99253,99253
Дальний Восток,94,94
Северо-Запад,266,266
Центр,387,387


## Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [76]:
#создаем переменную список years
years = list(range(1950,2010,1))
years[0:5]

[1950, 1951, 1952, 1953, 1954]

In [77]:
#Возьмем датафрейм из первого задания, с уже посчитанной средней оценкой для фильма
data_movies.head()

,movieId,rating,class,title,genres
0,1,3.9,средний,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,3.4,средний,Jumanji (1995),Adventure|Children|Fantasy
2,3,3.2,средний,Grumpier Old Men (1995),Comedy|Romance
3,4,2.4,средний,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,3.3,средний,Father of the Bride Part II (1995),Comedy


In [78]:
def production_year(row):
    for year in years:
        if str(year) in row.title:
            return year
    return 1900

In [79]:
data_movies['year'] = data_movies.apply(production_year, axis = 1 )
data_movies.head()

,movieId,rating,class,title,genres,year
0,1,3.9,средний,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,3.4,средний,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,3.2,средний,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,2.4,средний,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,3.3,средний,Father of the Bride Part II (1995),Comedy,1995


In [80]:
data_movies.groupby('year').mean()[['rating']].round(1).sort_values('rating',ascending = False).head(20)

,rating
year,
1951,3.9
1962,3.8
1952,3.8
1953,3.7
1973,3.7
1957,3.7
1976,3.6
1972,3.6
1971,3.6
